# Finding Best Location to open a Indian Restaurant in Miami, FL

In [3]:
# importing all neccessary dependencies 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
import requests
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize #for tranforming JSON file into a pandas dataframe

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [6]:
!pip install geopy
from geopy.geocoders import Nominatim # converting an address into latitude and longitude values

In [7]:
# install either version 3.4.2 of lxml or 4.4.2. Restart the kernel after installing if wikipedia table is not extracted
!pip install lxml==4.4.2

     |████████████████████████████████| 5.8MB 5.8MB/s eta 0:00:01
  Found existing installation: lxml 4.3.1
    Uninstalling lxml-4.3.1:
      Successfully uninstalled lxml-4.3.1


# #Scraping the List of Neighborhoods of Miami from Wikipedia and Preparing a Dataframe

In [11]:
url  = "https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [13]:
df_Miami = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_Miami.head(5)

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257


In [14]:
# Dropping unncessary column from the wikipedia table
df=df_Miami.drop(['Demonym', 'Population2010','Population/Km²','Sub-neighborhoods'], axis=1)
df.head()

,Neighborhood,Coordinates
0,Allapattah,25.815-80.224
1,Arts & Entertainment District,25.799-80.190
2,Brickell,25.758-80.193
3,Buena Vista,25.813-80.192
4,Coconut Grove,25.712-80.257


In [15]:
#Dropoing the the rows on which the Coordinates is empty
df.dropna(subset=['Coordinates'], inplace=True)

In [16]:
#Identifying Neighborhood is empty but coordinates exists
blank_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is blank: {}'.format(blank_neighborhood))

Number of rows on which Neighborhood column is blank: 0


In [17]:
# Splitting the coordinates column into Latitude and Longitude + Dropping the coordinates column
df['Latitude']= df['Coordinates'].str[0:6]
df['Longitude']=df['Coordinates'].str[6:13]
df.drop(columns =["Coordinates"], inplace = True) 
df.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


In [18]:
#Making sure that Latitude and Longitude columns are of numeric type
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

In [19]:
# Finding the shape of the dataset
print('The shape of the dataset is:',df.shape)

The shape of the dataset is: (24, 3)


In [20]:
# Getting the Latitude and Longitude of Miami city
address = 'Miami City, FL'

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Miami City are 25.7742658, -80.1936589.


In [22]:
# Creating a map of city of Miami showing neighborhoods
import folium
map_Miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# adding markers to map
for lat, lng, neighborhood in zip (df['Latitude'], 
                                  df['Longitude'], 
                                  df['Neighborhood']):
  
 folium.CircleMarker(
        [lat, lng],
        radius=5,
        fill_opacity=0.7).add_to(map_Miami)  
    
map_Miami

Exploring venues of Miami using Four Square API

In [23]:
# Feeding the Square Credentials to use
CLIENT_ID = 'OFEPYSMADPF55SSPVFKK4TBOGESZW3GZLTAYOTDYTZAWZHBY' # your Foursquare ID
CLIENT_SECRET = 'OLLDVDLQO0JXYRZBQMVESD4G0DAOFTDB1DLMCU5VBFWHUUUZ' # your Foursquare Secret
VERSION = '20200130' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OFEPYSMADPF55SSPVFKK4TBOGESZW3GZLTAYOTDYTZAWZHBY
CLIENT_SECRET:OLLDVDLQO0JXYRZBQMVESD4G0DAOFTDB1DLMCU5VBFWHUUUZ


In [24]:
# Exploring the latitude and longitude of a certain neighborhood of Miami 
N_latitude = df.loc[22, 'Latitude'] # Neighborhood latitude value
N_longitude = df.loc[22, 'Longitude'] # Neighborhood longitude value

N_name = df.loc[2, 'Neighborhood'] # Neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(N_name, 
                                                               N_latitude, 
                                                               N_longitude))

Latitude and longitude values of Brickell are 25.736, -80.155.


In [25]:
# setting up result limit and defining radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    N_latitude, 
    N_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json() # make the GET request

In [26]:
# function for finding the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
# Finding the venues of the neighborhood and cleaning the Json file

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flattening JSON

# filtering columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1) # filtering the category for each row

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns] # Preparing columns

nearby_venues.head()

,name,categories,lat,lng
0,Virginia Key Beach Park,Park,25.737444,-80.162234
1,WetLab,Dive Bar,25.731700,-80.163168
2,Key Biscayne Traffic Detail,Moving Target,25.729902,-80.158250
3,Party's by Pat,Food,25.728100,-80.156403
4,SALT Waterfront Restaurant,Cafeteria,25.731698,-80.163226


In [28]:
# Number of venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [29]:
## Exploring the Neighborhoods of Miami
#Getting venues for all neighborhoods of Miami
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
Miami_venues = getNearbyVenues(names=df['Neighborhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude'])

Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Midtown
Overtown
Park West
The Roads
Upper Eastside
Venetian Islands
Virginia Key
West Flagler
Wynwood


In [31]:
Miami_venues.shape

(1390, 7)

In [33]:
#Number of venues per neighborhood of Miami
Miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allapattah,14,14,14,14,14,14
Arts & Entertainment District,100,100,100,100,100,100
Brickell,100,100,100,100,100,100
Buena Vista,100,100,100,100,100,100
Coconut Grove,10,10,10,10,10,10
Coral Way,24,24,24,24,24,24
Design District,100,100,100,100,100,100
Downtown,100,100,100,100,100,100
Edgewater,100,100,100,100,100,100


In [34]:
#finding the list of categories
Miami_venues['Venue Category'].unique()[:100]

array(['Nightclub', 'Discount Store', 'Video Store', 'Grocery Store',
       'Park', 'Print Shop', 'Food & Drink Shop', 'Cosmetics Shop',
       'Spanish Restaurant', 'Bakery', 'Home Service', 'Cupcake Shop',
       'Wine Shop', 'Spa', 'Restaurant', 'Sandwich Place',
       'French Restaurant', 'Pizza Place', 'Gym', 'Ice Cream Shop',
       'Gun Range', 'Paper / Office Supplies Store', 'Beer Garden',
       'Donut Shop', 'Cuban Restaurant', 'Seafood Restaurant',
       'Salon / Barbershop', 'Moving Target', 'Brewery',
       'Tapas Restaurant', 'New American Restaurant', 'Art Gallery',
       'Peruvian Restaurant', 'Gym / Fitness Center', "Men's Store",
       'Empanada Restaurant', 'Burger Joint', 'Mexican Restaurant',
       'Breakfast Spot', 'Dessert Shop', 'Poke Place', 'Bar',
       'Smoothie Shop', 'Juice Bar', 'Asian Restaurant',
       'Indie Movie Theater', 'Coffee Shop', 'Neighborhood',
       'Cocktail Bar', 'South American Restaurant',
       'Residential Building (Apartmen

In [35]:
# Exploring the results containing the word "Indian Restaurants"
"Indian Restaurant" in Miami_venues['Venue Category'].unique()

True

In [36]:

## Analzing Each Neighborhood
# one hot encoding
Miami_onehot = pd.get_dummies(Miami_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column to dataframe
Miami_onehot['Neighborhood'] = Miami_venues['Neighborhood'] 

# moving neighborhood column to the first column
fixed_columns = [Miami_onehot.columns[-1]] + list(Miami_onehot.columns[:-1])
Miami_onehot = Miami_onehot[fixed_columns]


Miami_onehot.head()

,Yoga Studio,Accessories Store,Airport Service,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,Optical Shop,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,

In [37]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Miami_grouped = Miami_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(Miami_grouped.shape)
Miami_grouped

(24, 202)


,Neighborhood,Yoga Studio,Accessories Store,Airport Service,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Optical Shop,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allapattah,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.071429,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.071429,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.

In [40]:
len(Miami_grouped[Miami_grouped["Indian Restaurant"] > 0])

6

In [41]:
M_grouped = Miami_grouped[["Neighborhood","Indian Restaurant"]]
M_grouped.head()

,Neighborhood,Indian Restaurant
0,Allapattah,0.00
1,Arts & Entertainment District,0.00
2,Brickell,0.00
3,Buena Vista,0.02
4,Coconut Grove,0.00


K-Mean Clustring

In [42]:
# set number of clusters
toclusters = 3

to_clustering = M_grouped.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 2], dtype=int32)

In [43]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Cluster = M_grouped.copy()

# add clustering labels
Cluster["Cluster Labels"] = kmeans.labels_

In [44]:
Cluster.rename(columns={"Neighborhood": "Neighborhood"}, inplace=True)
Cluster.head(20)

,Neighborhood,Indian Restaurant,Cluster Labels
0,Allapattah,0.000000,1
1,Arts & Entertainment District,0.000000,1
2,Brickell,0.000000,1
3,Buena Vista,0.020000,0
4,Coconut Grove,0.000000,1
5,Coral Way,0.000000,1
6,Design District,0.020000,0
7,Downtown,0.000000,1
8,Edgewater,0.010000,0
9,Flagami,0.034483,2


In [45]:
# merging Miami_grouped with Cluster Labels data to add latitude/longitude for each neighborhood
Cluster = Cluster.join(Miami_venues.set_index("Neighborhood"), on="Neighborhood")

print(Cluster.shape)
Cluster.head(5)

(1390, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allapattah,0.0,1,25.815,-80.224,Club Tipico Dominicano,25.809557,-80.218593,Nightclub
0,Allapattah,0.0,1,25.815,-80.224,Family Dollar,25.807208,-80.223503,Discount Store
0,Allapattah,0.0,1,25.815,-80.224,Redbox,25.808122,-80.224456,Video Store
0,Allapattah,0.0,1,25.815,-80.224,Winn-Dixie,25.808179,-80.224911,Grocery Store
0,Allapattah,0.0,1,25.815,-80.224,Charles Hadley Pool,25.819565,-80.216753,Park


In [46]:
# sorting the output by Cluster Labels
print(Cluster.shape)
Cluster.sort_values(["Cluster Labels"], inplace=True)
Cluster.head()

(1390, 9)


,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23,Wynwood,0.01,0,25.804,-80.199,Macaya Gallery,25.810540,-80.198363,Art Gallery
6,Design District,0.02,0,25.813,-80.193,Guarapo Organic Juice Bar - 36th Street,25.810394,-80.198820,Juice Bar
6,Design District,0.02,0,25.813,-80.193,Mihos Holistic Space,25.813375,-80.199637,Dance Studio
6,Design District,0.02,0,25.813,-80.193,El Bori Foodtruck,25.811693,-80.199550,Food Truck
6,Design District,0.02,0,25.813,-80.193,E. Albert Pallor Park,25.813021,-80.186251,Park


In [47]:
# Loading Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11,tiles='cartodbpositron')

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [49]:

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster['Neighborhood Latitude'], Cluster['Neighborhood Longitude'], Cluster['Neighborhood'], Cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

In [50]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [51]:
#Cluster 0
Cluster.loc[Cluster['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23,Wynwood,0.01,0,25.804,-80.199,Macaya Gallery,25.810540,-80.198363,Art Gallery
6,Design District,0.02,0,25.813,-80.193,Guarapo Organic Juice Bar - 36th Street,25.810394,-80.198820,Juice Bar
6,Design District,0.02,0,25.813,-80.193,Mihos Holistic Space,25.813375,-80.199637,Dance Studio
6,Design District,0.02,0,25.813,-80.193,El Bori Foodtruck,25.811693,-80.199550,Food Truck
6,Design District,0.02,0,25.813,-80.193,E. Albert Pallor Park,25.813021,-80.186251,Park
6,Design District,0.02,0,25.813,-80.193,Middle Park,25.811369,-80.186202,Park
6,Design District,0.02,0,25.813,-80.193,Pandorée,25.810211,-80.199511,Bakery
6,Design District,0.02,0,25.813,-80.193,Lotus House Thrift,25.810542,-80.199773,Thrift / Vintage Store
8,Edgewater,0.01,0,25.802,-80.190,NOA Catering,25.803114,-80.190942,Restaurant
8,Edgewater,0.01,0,25.802,-80.190,Enriqueta's Sandwich Shop,25.804086,-80.191279,Cuban Restaurant


In [56]:
#Cluster 1
Cluster.loc[Cluster['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
22,West Flagler,0.0,1,25.775,-80.243,Bolivian Restaurant Las Americas,25.779578,-80.240798,South American Restaurant
22,West Flagler,0.0,1,25.775,-80.243,Raspados Nicaraguenses,25.772618,-80.236857,Latin American Restaurant
22,West Flagler,0.0,1,25.775,-80.243,Grand Paris,25.779059,-80.243984,Bakery
22,West Flagler,0.0,1,25.775,-80.243,Shell,25.772624,-80.239278,Gas Station
22,West Flagler,0.0,1,25.775,-80.243,Miami Smokers,25.775735,-80.239385,Comfort Food Restaurant
22,West Flagler,0.0,1,25.775,-80.243,Fritanga Caña Brava,25.779708,-80.234640,Latin American Restaurant
22,West Flagler,0.0,1,25.775,-80.243,Miami Dade County Auditorium,25.773074,-80.241118,Concert Hall
22,West Flagler,0.0,1,25.775,-80.243,Taqueria El Carnal 3,25.772537,-80.238386,Mexican Restaurant
22,West Flagler,0.0,1,25.775,-80.243,Don Pan,25.778408,-80.239545,Bakery
22,West Flagler,0.0,1,25.775,-80.243,Salmon & Salmon Restaurant,25.779733,-80.242164,Latin American Restaurant


In [55]:
#Cluster 2
Cluster.loc[Cluster['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,Flagami,0.034483,2,25.762,-80.316,Walgreens Liquor,25.769852,-80.320002,Liquor Store
9,Flagami,0.034483,2,25.762,-80.316,T-Mobile,25.762570,-80.308706,Mobile Phone Shop
9,Flagami,0.034483,2,25.762,-80.316,Canton Palace,25.761934,-80.325833,Chinese Restaurant
9,Flagami,0.034483,2,25.762,-80.316,Walmart Neighborhood Market,25.764574,-80.308947,Grocery Store
9,Flagami,0.034483,2,25.762,-80.316,Pan de Bono,25.762874,-80.317429,Bakery
9,Flagami,0.034483,2,25.762,-80.316,El Gallego Restaurant,25.763045,-80.311695,Spanish Restaurant
9,Flagami,0.034483,2,25.762,-80.316,Bahama's Fish Market & Restaurant,25.762608,-80.312045,Seafood Restaurant
9,Flagami,0.034483,2,25.762,-80.316,Pollo Tropical,25.763535,-80.309072,Fried Chicken Joint
9,Flagami,0.034483,2,25.762,-80.316,Macy's- Dadeland Mall,25.762272,-80.318153,Department Store
9,Flagami,0.034483,2,25.762,-80.316,Allegro Music,25.763306,-80.306797,Hobby Shop
